<a href="https://colab.research.google.com/github/franfram/Transformers-nlp/blob/main/nlp-pipeline-app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

# https://stackoverflow.com/questions/51666374/how-to-remove-strings-present-in-a-list-from-a-column-in-pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import AutoTokenizer

# Load tokenizer
checkpoint = "CenIA/distillbert-base-spanish-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

vocab = tokenizer.get_vocab()



In [6]:
import pandas as pd
import re
url = 'https://raw.githubusercontent.com/franfram/Transformers-nlp/main/lista-pictogramas.csv'
data = pd.read_csv(url, header=None)
pictograms = data.melt().value.tolist()

pictograms_cleaned = [x for x in pictograms if str(x) != 'nan']
pictograms_cleaned

pictograms_separated = []
for pict in range(len(pictograms_cleaned)): 
  pictograms_separated.append(re.findall(r"[\w']+", pictograms_cleaned[pict]))


# flatten list
def flatten(l):
    return [item for sublist in l for item in sublist]

flat_list = set(flatten(pictograms_separated))
words_to_delete = list(flat_list)


# save to list
pd.DataFrame(words_to_delete).to_csv('pictograms_cleaned.csv')


Load the data, melt into a single column, separate the pictograms that contain more than one word, remove duplicates. 

create bad_words_ids for text generation

In [7]:


bad_words_ids = list(vocab.keys())



def create_bad_words_ids(bad_words_ids, words_to_delete):
  for pictogram in range(len(words_to_delete)):
    if words_to_delete[pictogram] in bad_words_ids:
      bad_words_ids.remove(words_to_delete[pictogram])
  return bad_words_ids

bad_words_ids = create_bad_words_ids(bad_words_ids=bad_words_ids, words_to_delete=words_to_delete)



# Create lists of lists from list, which is what is required by the generator method
def list_of_lists(lst):
    return [[el] for el in lst]

bad_words_ids_listoflist = list_of_lists(bad_words_ids)
print(bad_words_ids_listoflist)


[['condición'], ['tratada'], ['vocales'], ['retrasar'], ['aplicada'], ['chocola'], ['discrim'], ['ville'], ['paraná'], ['schmidt'], ['atraparlo'], ['benedic'], ['##at'], ['modificado'], ['##biliario'], ['[unused423]'], ['##%'], ['germán'], ['##presu'], ['ofrenda'], ['##tándote'], ['mix'], ['rocío'], ['research'], ['topo'], ['##idu'], ['[unused648]'], ['avenidas'], ['recep'], ['financiera'], ['pasos'], ['##leto'], ['##cina'], ['##ked'], ['manicom'], ['emergentes'], ['comen'], ['gene'], ['enunciado'], ['portales'], ['magic'], ['##itaciones'], ['capacidad'], ['cancer'], ['superfici'], ['controlada'], ['##riéndose'], ['##fl'], ['regu'], ['pelu'], ['etno'], ['perdió'], ['reaccionar'], ['carm'], ['añad'], ['detall'], ['billetes'], ['##risas'], ['ácidos'], ['apen'], ['inventé'], ['##posición'], ['fecha'], ['monje'], ['morales'], ['editar'], ['titularidad'], ['originó'], ['compradores'], ['dero'], ['##tidor'], ['prome'], ['album'], ['##ches'], ['##pool'], ['relie'], ['in'], ['##ional'], ['abri

Text generation with bad_words_ids (not working atm)
REMEMBER THAT YOU ARE USING THE VOCAB FROM A FILL MASK MODEL, NOT THE VOCAB FROM THIS TEXT GENERATION MODEL. YOU HAVE TO get the vocab of "PlanTL-GOB-ES/gpt2-base-bne" and probably make a function of everything to make a pipeline. 

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/gpt2-base-bne")

model = AutoModelForCausalLM.from_pretrained("PlanTL-GOB-ES/gpt2-base-bne")

generator = pipeline("text-generation", model=model,tokenizer=tokenizer)
generator(
    "Quisiera saber que va a suceder",
    max_length=20,
    num_return_sequences=5,
    bad_words_ids=bad_words_ids_listoflist
)

Downloading:   0%|          | 0.00/681 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

ValueError: ignored

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 '.',
 'Ġthe',
 ',',
 'Ġto',
 'Ġand',
 'Ġof',
 'Ġa',
 'Ġin',
 '-',
 'Ġfor',
 'Ġthat',
 'Ġon',
 'Ġis',
 'âĢ',
 "'s",
 'Ġwith',
 'ĠThe',
 'Ġwas',
 'Ġ"',
 'Ġat',
 'Ġit',
 'Ġas',
 'Ġsaid',
 'Ļ',
 'Ġbe',
 's',
 'Ġby',
 'Ġfrom',
 'Ġare',
 'Ġhave',
 'Ġhas',
 ':',
 'Ġ(',
 'Ġhe',
 'ĠI',
 'Ġhis',
 'Ġwill',
 'Ġan',
 'Ġthis',
 ')',
 'ĠâĢ',
 'Ġnot',
 'Ŀ',
 'Ġyou',
 'ľ',
 'Ġtheir',
 'Ġor',
 'Ġthey',
 'Ġwe',
 'Ġbut',
 'Ġwho',
 'Ġmore',
 'Ġhad',
 'Ġbeen',
 'Ġwere',
 'Ġabout',
 ',"',
 'Ġwhich',
 'Ġup',
 'Ġits',
 'Ġcan',
 'Ġone',
 'Ġout',
 'Ġalso',
 'Ġ$',
 'Ġher',
 'Ġall',
 'Ġafter',
 '."',
 '/',
 'Ġwould',
 "'t",
 'Ġyear',
 'Ġwhen',
 'Ġfirst',
 'Ġshe',
 'Ġtwo',
 'Ġover',
 'Ġpeople',
 'ĠA',
 'Ġour',
 'ĠIt',
 'Ġtime',
 'Ġthan',
 'Ġinto',
 'Ġthere',
 't',
 'ĠHe',
 'Ġnew',
 'ĠâĢĶ',
 'Ġlast',
 'Ġjust',
 'ĠIn',
 'Ġother',
 'Ġso',
 'Ġwhat']

In [37]:
from transformers import pipeline
top_k = 200 #@param {type: "integer"}

unmasker = pipeline("fill-mask", model="CenIA/distillbert-base-spanish-uncased")
text = 'quiero comer un ' #@param {type: "string"}
text += ' [MASK]'

unmasked = unmasker(text, top_k=top_k)


unmasked



# list = []
# for unmask in unmasked: 
#   list.append(unmask["token_str"])

# list



# def create_bad_words_ids(bad_words_ids, words_to_delete):
#   for pictogram in range(len(words_to_delete)):
#     if words_to_delete[pictogram] in bad_words_ids:
#       bad_words_ids.remove(words_to_delete[pictogram])
#   return bad_words_id

# output = {}
# for unmask in unmasked:
#   output[unmask["token_str"]] = unmask["score"]

# output
# unmasker = pipeline('fill-mask', model="CenIA/distillbert-base-spanish-uncased")

# def fill_mask(text):
#   text += " [MASK]"
#   unmasked = unmasker(text, top_k=8)
#   output = {}
#   for unmask in unmasked:
#     output[unmask["token_str"]] = unmask["score"]
#   return output

# examples = [["quiero comer "], ["me gustaría ir a"]]


[{'score': 0.1072106584906578,
  'token': 11273,
  'token_str': 'helado',
  'sequence': 'quiero comer un helado'},
 {'score': 0.07168658077716827,
  'token': 28110,
  'token_str': 'filete',
  'sequence': 'quiero comer un filete'},
 {'score': 0.06784360855817795,
  'token': 12232,
  'token_str': 'huevo',
  'sequence': 'quiero comer un huevo'},
 {'score': 0.06588539481163025,
  'token': 8633,
  'token_str': 'pastel',
  'sequence': 'quiero comer un pastel'},
 {'score': 0.04792048782110214,
  'token': 9689,
  'token_str': 'pollo',
  'sequence': 'quiero comer un pollo'},
 {'score': 0.046015117317438126,
  'token': 10422,
  'token_str': 'plato',
  'sequence': 'quiero comer un plato'},
 {'score': 0.03697066009044647,
  'token': 1819,
  'token_str': 'poco',
  'sequence': 'quiero comer un poco'},
 {'score': 0.032784394919872284,
  'token': 9160,
  'token_str': 'cerdo',
  'sequence': 'quiero comer un cerdo'},
 {'score': 0.024693995714187622,
  'token': 14532,
  'token_str': 'conejo',
  'sequence

# Pipeline

We will first def a function to extract the predicted words (for later, also the score with a dict or something else). Then we will def a function to delete from the predicted words those who are not in our pictograms. Finally, we will def a function to get the k most likely words. The pipeline will be model_output -> extract_predicted_words() -> filter_according_to_pictograms() -> get_top_k_predictions()

Def extract_predicted_words

In [38]:

predicted_words = []
def extract_predicted_words(unmasked, predicted_words):
  for unmask in unmasked: 
    predicted_words.append(unmask['token_str'])# Maybe the problem is in unmask["token_str"]??
  return predicted_words


predicted_words = extract_predicted_words(unmasked, predicted_words)

predicted_words, len(predicted_words)



(['helado',
  'filete',
  'huevo',
  'pastel',
  'pollo',
  'plato',
  'poco',
  'cerdo',
  'conejo',
  'pavo',
  'pez',
  'postre',
  'trago',
  'refresco',
  'pescado',
  'pedazo',
  'perrito',
  'perro',
  'trozo',
  'chocolate',
  'gato',
  'gusano',
  'almuerzo',
  'pan',
  'caramelo',
  'poquito',
  'rato',
  'desayuno',
  '[UNK]',
  'dulce',
  'hueso',
  'caballo',
  'monton',
  'panque',
  'cafe',
  'asado',
  'queso',
  'cono',
  '.',
  'cangrejo',
  'pizza',
  'pato',
  'cigarro',
  'puerco',
  'cordero',
  'vaso',
  '"',
  'bebe',
  'animal',
  'ave',
  'yogur',
  'bis',
  'insecto',
  'par',
  'chicle',
  'bocad',
  'cigarrillo',
  'oso',
  'taco',
  'buen',
  'hamburguesa',
  'coco',
  'te',
  'bicho',
  'bol',
  'ta',
  'solo',
  'sushi',
  'dia',
  'restaurante',
  'man',
  'gallo',
  'leon',
  'elefante',
  'cereal',
  'pene',
  'ciervo',
  'caracol',
  'rico',
  'fresco',
  '-',
  'cuchillo',
  'camion',
  'galleta',
  'cubo',
  'gatito',
  'hongo',
  'pajarito',
  'ra

Def filter_according_to_pictograms()

In [39]:

def filter_according_to_pictograms(predicted_words): 
  predicted_words[:] = [word for word in predicted_words if word in pictogram_list]
  return predicted_words

filtered_pictograms = filter_according_to_pictograms(predicted_words)

print(filtered_pictograms)



['helado', 'huevo', 'pollo', 'poco', 'cerdo', 'conejo', 'pez', 'pescado', 'perro', 'gato', 'gusano', 'pan', 'caramelo', 'desayuno', 'dulce', 'hueso', 'caballo', 'cafe', 'queso', 'cangrejo', 'pizza', 'pato', 'oso', 'buen', 'hamburguesa', 'te', 'bicho', 'dia', 'restaurante', 'gallo', 'leon', 'elefante', 'pene', 'caracol', 'camion', 'dinosaurio', 'arbol', 'hombre', 'tigre', 'chico', 'tomate', 'grande', 'regalo', 'manzana', 'largo', 'libro', 'arroz', 'papa', 'ensalada', 'amigo', 'naranja', 'tronco', 'margarita', 'sopa', 'rio', 'lobo', 'nuevo', 'mono', 'culo', 'no', 'taxi', 'pie', 'globo', 'pis', 'cuento', 'torta', 'mar']


def get_top_k_predictions (words in `unmasked` are already ordered according to probability, so we simply need to extract the top k predictions)

In [40]:
def get_top_k_predictions(filtered_pictograms, k):
  if len(filtered_pictograms) > k:
    top_k_predictions = filtered_pictograms[:k]
  else: 
    top_k_predictions = filtered_pictograms
  return top_k_predictions


top_k_predictions = get_top_k_predictions(filtered_pictograms, 8)

top_k_predictions


['helado', 'huevo', 'pollo', 'poco', 'cerdo', 'conejo', 'pez', 'pescado']